# New QA tests

We are going to make new QA tests, by reconstructing bigger trees.

In [1]:
import sys
sys.path.append('../')
from qa_functions import *
from colorama import Fore
import time
import networkx as nx
import random
import re
import os

In [ ]:
file_names = os.listdir('alignments')

# Remove the folder test
file_names = file_names[:-1]

for folder in file_names:
    if int(folder) > 31:
        files = os.listdir(f'alignments/{folder}')
        for file in files:
            
            print(f'alignments/{folder}/{file}')
            # Load the sequences and create simmilarity matrix
            sequences,labels = load_sequences(f'./alignments/{folder}/{file}',format='fasta')
            distance_matrix = compute_distance_matrix(sequences)

            # Save the distance matrix
            file_ext = re.search(r'_([0-9]+)\.',file).group(1)
            matrix_file = new_file = f'matrix_{file_ext}'
            np.save(f'./matrices/{folder}/{matrix_file}',distance_matrix)
            
            # Begin NMcutQA
            timer = Timer(0.0)
            tree_qa = phylo_tree(distance_matrix,timer=timer)
            
            with open('timer.csv','a') as fp:
                fp.write(f'{folder},{file},{timer.value}\n')
            
            new_file = f'qa_tree_{file_ext}.newick'
            tree_qa.create_newick_file(f'./trees/{folder}/{new_file}')
            print(f'Finished {file} in {timer.value}ms')

            # Erase memory
            del sequences
            del labels
            del distance_matrix
            del tree_qa

In [2]:
file = 'assembled_sequences_49038.fasta'
folder = '43'

print(f'alignments/{folder}/{file}')
# Load the sequences and create simmilarity matrix
sequences,labels = load_sequences(f'../alignments/{folder}/{file}',format='fasta')
distance_matrix = compute_distance_matrix(sequences)

# Save the distance matrix
file_ext = re.search(r'_([0-9]+)\.',file).group(1)
matrix_file = new_file = f'matrix_{file_ext}'
np.save(f'../matrices/{folder}/{matrix_file}',distance_matrix)

# Begin NMcutQA
timer = Timer(0.0)
tree_qa = phylo_tree(distance_matrix,timer=timer)

with open('../metrics/timer.csv','a') as fp:
    fp.write(f'{folder},{file},{timer.value}\n')

new_file = f'qa_tree_{file_ext}.newick'
tree_qa.create_newick_file(f'../trees/{folder}/{new_file}')
print(f'Finished {file} in {timer.value}ms')

# Erase memory
del sequences
del labels
del distance_matrix
del tree_qa

alignments/43/assembled_sequences_49038.fasta
Finished assembled_sequences_49038.fasta in 2446.8066400000002ms


In [8]:
with open(f'../trees/{folder}/{new_file}','r') as file:
    qa_tree = file.read()

with open(f'../trees/{folder}/tree_best_{file_ext}.newick','r') as file:
    biotree = file.read()
biotree = re.sub(r'taxon([0-9]+)',lambda match: str(int(match.group(1)) - 1),biotree)
biotree = re.sub(r':[0-9]\.[0-9]+(e-[0-9]+)*',r'',biotree)

dist = treecmp(qa_tree,biotree)

print(f'The Robinson-Foulds distance bewteen the Ncut with QA tree and the Neighbor-Joining tree is \033[1m{dist}\033[22m')

The Robinson-Foulds distance bewteen the Ncut with QA tree and the Neighbor-Joining tree is 25.0


We have generated alignments for trees from size 100 to 150. Remember, the expected number of subproblems is given by the expression:
$$\mu(x)=0.35x\cdot \log_2{10.82x}.$$

Given that a problem usually takes around 3 seconds to be enter and run in the remote QPU, the expected total time for the execution will be:

In [16]:
def f(x):
    return 0.35*x*np.log2(10.82*x)

x = np.arange(100,151,2)
print(f'The full execution will take approximately {round(np.sum(f(x)*3/60)/60,3)} hours')

The full execution will take approximately 9.87 hours


We can run the program here, but it should be run in CESGA or similar.

In [ ]:
file_names = os.listdir('../alignments')

# Remove the folder test
file_names = file_names[:-1]

for folder in file_names:
    if int(folder) > 99 and int(folder) < 125:
        files = os.listdir(f'../alignments/{folder}')
        for file in files:
            
            print(f'alignments/{folder}/{file}')
            # Load the sequences and create simmilarity matrix
            sequences,labels = load_sequences(f'../alignments/{folder}/{file}',format='fasta')
            distance_matrix = compute_distance_matrix(sequences)

            # Save the distance matrix
            file_ext = re.search(r'_([0-9]+)\.',file).group(1)
            matrix_file = new_file = f'matrix_{file_ext}'
            np.save(f'../matrices/{folder}/{matrix_file}',distance_matrix)
            print(f'Alignment file generated, generating the phylogenetic tree...')
            # Begin NMcutQA
            timer = Timer(0.0)
            tree_qa = phylo_tree(distance_matrix,timer=timer)

            with open('../metrics/timer.csv','a') as fp:
                fp.write(f'{folder},{file},{timer.value}\n')

            new_file = f'qa_tree_{file_ext}.newick'
            tree_qa.create_newick_file(f'../trees/{folder}/{new_file}')
            print(f'Finished {file} in {timer.value}ms\n---------------------------------------------------------\n')

            # Erase memory
            del sequences
            del labels
            del distance_matrix
            del tree_qa

alignments/100/assembled_sequences_53321.fasta
Alignment file generated, generating the phylogenetic tree...
Finished assembled_sequences_53321.fasta in 13510.93888ms
---------------------------------------------------------

alignments/102/assembled_sequences_30931.fasta
Alignment file generated, generating the phylogenetic tree...


In [3]:
file_ext = '53321'

with open(f'../trees/100/qa_tree_{file_ext}.newick','r') as file:
    qa_tree = file.read()

with open(f'../trees/100/tree_best_{file_ext}.newick','r') as file:
    biotree = file.read()
biotree = re.sub(r'taxon([0-9]+)',lambda match: str(int(match.group(1)) - 1),biotree)
biotree = re.sub(r':[0-9]\.[0-9]+(e-[0-9]+)*',r'',biotree)

dist = treecmp(qa_tree,biotree)

print(f'The Robinson-Foulds distance bewteen the Ncut with QA tree and the Neighbor-Joining tree is \033[1m{dist}\033[22m')

The Robinson-Foulds distance bewteen the Ncut with QA tree and the Neighbor-Joining tree is 5.15


In [4]:
from ete3 import Tree
tree1 = Tree(qa_tree)
tree2 = Tree(biotree)

cmp = tree1.compare(tree2,unrooted=True)
print(cmp)

{'rf': 184.0, 'max_rf': 194.0, 'ref_edges_in_source': 0.5306122448979592, 'source_edges_in_ref': 0.5306122448979592, 'effective_tree_size': 100, 'norm_rf': 0.9484536082474226, 'treeko_dist': 'NA', 'source_subtrees': 1, 'common_edges': {(('0', '1', '10', '11', '12', '13', '14', '15', '16', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '6', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '7', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '8', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '9', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99'), ('17',)), (('0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '